In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from data import merged_data, get_education_indicators, create_indicator_levels, get_education_data, get_hlo_data, DATA_PATH, get_education_meta, get_gdp_data

In [2]:
edu = get_education_data()
hlo = get_hlo_data()

edu.head()

completed_education_rate  completion_rate  \
country_code year   level gender                                              
ABW          1960.0 total total                        NaN              NaN   
             1961.0 total total                        NaN              NaN   
             1962.0 total total                        NaN              NaN   
             1963.0 total total                        NaN              NaN   
             1964.0 total total                        NaN              NaN   

                                  completion_rate_lower  completion_rate_post  \
country_code year   level gender                                                
ABW          1960.0 total total                     NaN                   NaN   
             1961.0 total total                     NaN                   NaN   
             1962.0 total total                     NaN                   NaN   
             1963.0 total total                     NaN                   NaN   
             1964.0 total total                     NaN                   NaN   

                                  completion_rate_upper  \
country_code year   level gender                          
ABW          1960.0 total total                     NaN   
             1961.0 total total                     NaN   
             1962.0 total total                     NaN   
             1963.0 total total                     NaN   
             1964.0 total total                     NaN   

                                  compulsory_education_duration  \
country_code year   level gender                                  
ABW          1960.0 total total                             NaN   
             1961.0 total total                             NaN   
             1962.0 total total                             NaN   
             1963.0 total total                             NaN   
             1964.0 total total                             NaN   

                                  education_duration  education_expenditure  \
country_code year   level gender                                              
ABW          1960.0 total total                  NaN                    NaN   
             1961.0 total total                  NaN                    NaN   
             1962.0 total total                  NaN                    NaN   
             1963.0 total total                  NaN                    NaN   
             1964.0 total total                  NaN                    NaN   

                                  education_expenditure_gdp_rate  \
country_code year   level gender                                   
ABW          1960.0 total total                              NaN   
             1961.0 total total                              NaN   
             1962.0 total total                              NaN   
             1963.0 total total                              NaN   
             1964.0 total total                              NaN   

                                  education_expenditure_rate  ...  \
country_code year   level gender                              ...   
ABW          1960.0 total total                          NaN  ...   
             1961.0 total total                          NaN  ...   
             1962.0 total total                          NaN  ...   
             1963.0 total total                          NaN  ...   
             1964.0 total total                          NaN  ...   

                                  population_0_14  population_15_64  \
country_code year   level gender                                      
ABW          1960.0 total total         43.847190         53.669919   
             1961.0 total total         43.358346         54.056784   
             1962.0 total total         42.925745         54.383281   
             1963.0 total total         42.488756         54.710292   
             1964.0 total total         41.950133         55.119933   

               

In [3]:
df = merged_data()
df.isna().any()

completed_education_rate           True
completion_rate                    True
completion_rate_lower              True
completion_rate_post               True
completion_rate_upper              True
compulsory_education_duration      True
country_name                      False
education_duration                 True
education_expenditure              True
education_expenditure_gdp_rate     True
education_expenditure_rate         True
education_pupils                   True
expenditure_per_student            True
expenditure_rate                   True
gdppc                              True
income_group                      False
learning_outcome                   True
literacy_rate                      True
out_of_school_rate                 True
population                         True
population_0_14                    True
population_15_64                   True
pupil_teacher_ratio                True
region                            False
school_enrollment                  True


In [6]:
df[df.country_name.isna()]

completed_education_rate  \
country_code year   level     gender                             
ALB          2000.0 total     female                       NaN   
                              male                         NaN   
AUS          2000.0 total     female                       NaN   
                              male                         NaN   
             2003.0 total     female                       NaN   
...                                                        ...   
XKX          2015.0 secondary male                         NaN   
                              total                        NaN   
                    total     female                       NaN   
                              male                         NaN   
                              total                        NaN   

                                      completion_rate  completion_rate_lower  \
country_code year   level     gender                                           
ALB          2000.0 total     female              NaN                    NaN   
                              male                NaN                    NaN   
AUS          2000.0 total     female              NaN                    NaN   
                              male                NaN                    NaN   
             2003.0 total     female              NaN                    NaN   
...                                               ...                    ...   
XKX          2015.0 secondary male                NaN                    NaN   
                              total               NaN                    NaN   
                    total     female              NaN                    NaN   
                              male                NaN                    NaN   
                              total               NaN                    NaN   

                                      completion_rate_post  \
country_code year   level     gender                         
ALB          2000.0 total     female                   NaN   
                              male                     NaN   
AUS          2000.0 total     female                   NaN   
                              male                     NaN   
             2003.0 total     female                   NaN   
...                                                    ...   
XKX          2015.0 secondary male                     NaN   
                              total                    NaN   
                    total     female                   NaN   
                              male                     NaN   
                              total                    NaN   

                                      completion_rate_upper  \
country_code year   level     gender                          
ALB          2000.0 total     female                    NaN   
                              male                      NaN   
AUS          2000.0 total     female                    NaN   
                              male                      NaN   
             2003.0 total     female                    NaN   
...                                                     ...   
XKX          2015.0 secondary male                      NaN   
                              total                     NaN   
                    total     female                    NaN   
                              male                      NaN   
                              total                     NaN   

                                      compulsory_education_duration  \
country_code year   level     gender                                  
ALB          2000.0 total     female                            NaN   
                              male                              NaN   
AUS          2000.0 total     female                            NaN   
                              male                              NaN   
             2003.0 total     female                            NaN   
...           